# SPGR

In [1]:
import version_control

Last run on cashlab03-deb at 2015-03-13 14:00:28.973245

Phypno Version: 666ba4e19ef685b12751fd56592258eb41ecb2ac

Babel==1.3, Cython==0.22, Jinja2==2.7.3, Markdown==2.6.1, MarkupSafe==0.23, Pillow==2.7.0, PyOpenGL==3.1.0, Pygments==2.0.2, Sphinx==1.3, alabaster==0.7.2, beautifulsoup4==4.3.2, certifi==14.05.14, coverage==3.7.1, decorator==3.4.0, docutils==0.12, easydev==0.8.3, ipython==3.0.0, jsonschema==2.4.0, matplotlib==1.4.3, memory-profiler==0.32, mistune==0.5.1, networkx==1.9.1, nibabel==2.0.0, nitime==0.5, nose==1.3.4, numpy==1.9.2, numpydoc==0.5, ordereddict==1.1, pandas==0.15.2, pandocfilters==1.2.3, pep8==1.6.2, pockets==0.2.3, psutil==2.2.1, pycurl==7.19.5.1, pyflakes==0.8.1, pyparsing==2.0.3, pyqtgraph==0.9.10, python-dateutil==2.4.1, python-igraph==0.7.1-4, pytz==2014.10, pyzmq==14.5.0, rope-py3k==0.9.4-1, rpy2==2.5.6, scikit-learn==0.15.2, scipy==0.15.1, six==1.9.0, snowballstemmer==1.2.0, sphinx-rtd-theme==0.1.7, sphinxcontrib-napoleon==0.3.1, spyder==2.3.3, tornado==4.1

In [2]:
from spgr import lg
from logging import INFO
lg.setLevel(INFO)

## Read Sleep

### Selection of Patients

Recordings were obtained from six patients with drug-resistant epilepsy implanted with a semi-chronic grid to localize the seizure origin.
The grid consisted of platinum electrodes spaced 1cm apart on the pial surface.
The number of electrodes per patient was on average 81.67 (range 64--93).
Recordings were performed with standard clinical EEG monitoring equipment (XLTEK, Natus Medical Inc., Pleasanton, CA, USA) at 500 Hz, 512 Hz, or 1024 Hz.
The research was approved by the local institutional review board and electrode placement was determined solely by clinical criteria.
More information about patients' pathologies...

In [3]:
# define patients with grids
subj_with_grid_scalp = ('EM03', 'EM09', 'MG07', 'MG11', 'MG12', 'MG15', 'MG17', 
                        'MG29', 'MG31', 'MG32', 'MG33', 'MG37', 'MG42', 'MG45', 
                        'MG48', 'MG49', 'MG51', 'MG56', 'MG61', 'MG63', 'MG65', 'MG72')
subj_with_grid_only = ('MG28', 'MG67', )

# define patients to exclude
subj_with_many_SZ = ('MG49', 'MG74')
not_in_rec = ('MG12', )

all_subj = (set(subj_with_grid_scalp) | set(subj_with_grid_only)) - set(subj_with_many_SZ) - set(not_in_rec)

In [4]:
PERIOD = 'sleep'
STAGES = ('NREM2', )
MIN_DURATION = 60 * 60

In [5]:
from spgr.read_data import select_scores, save_data, REC_DIR
scores = select_scores(STAGES, MIN_DURATION, sorted(all_subj, reverse=True))

lg.info('\nPatients with sufficient recordings:')
lg.info('\t'.join(sorted(scores.keys())))

INFO:/home/gio/projects/spgr/scripts/spgr/read_data.py:MG72: duration not sufficient
INFO:/home/gio/projects/spgr/scripts/spgr/read_data.py:MG67: duration not sufficient
INFO:/home/gio/projects/spgr/scripts/spgr/read_data.py:MG65: duration not sufficient
INFO:/home/gio/projects/spgr/scripts/spgr/read_data.py:MG63 has MG63_eeg_xltek_sessA_d06_15_55_12_scores.xml
INFO:/home/gio/projects/spgr/scripts/spgr/read_data.py:MG61 has MG61_eeg_xltek_sessA_d03_07_58_11_scores.xml
INFO:/home/gio/projects/spgr/scripts/spgr/read_data.py:MG56: duration not sufficient
INFO:/home/gio/projects/spgr/scripts/spgr/read_data.py:MG51: duration not sufficient
INFO:/home/gio/projects/spgr/scripts/spgr/read_data.py:MG37 has MG37_eeg_xltek_sessA_d03_12_18_14_scores.xml
INFO:/home/gio/projects/spgr/scripts/spgr/read_data.py:MG33 has MG33_eeg_xltek_sessA_d09_09_01_22_scores.xml
INFO:/home/gio/projects/spgr/scripts/spgr/read_data.py:MG28: duration not sufficient
INFO:/home/gio/projects/spgr/scripts/spgr/read_data.py

In [6]:
good_subj = sorted(list(scores.keys()))

lg.info('Grid patients with scalp : ' + ', '.join(sorted(set(good_subj) & set(subj_with_grid_scalp))))
lg.info('Grid patients without scalp : ' + ', '.join(sorted(set(good_subj) & set(subj_with_grid_only))))
lg.info('Depth patients without scalp : ' + ', '.join(sorted(set(good_subj) - set(subj_with_grid_scalp) - set(subj_with_grid_only))))

INFO:/home/gio/projects/spgr/scripts/spgr/read_data.py:Grid patients with scalp : EM09, MG17, MG33, MG37, MG61, MG63
INFO:/home/gio/projects/spgr/scripts/spgr/read_data.py:Grid patients without scalp : 
INFO:/home/gio/projects/spgr/scripts/spgr/read_data.py:Depth patients without scalp : 


### Read Recordings

Periods that did not have any seizure in the preceding or following 12 hours were visually scored from the ECoG electrodes following the standard sleep stage classification [@Iber2007].
For each patient, we obtained on average 69.67 minutes (range 60--101.5) of NREM2 sleep that was included in the analysis.
Channels containing large interictal activity were excluded and the signal was referenced to the average (NOT ALWAYS).
Recordings were filtered at 0.5 Hz (high-pass filter) and at 50 Hz (low-pass filter) and subsequently downsampled to 256 Hz.

In [9]:
options = {'to_plot': False,
           'resample_freq': 256,
           'hp_filter': 0.5,
           'lp_filter': 50,
           }
chan_type = ('grid', 'strip')

In [10]:
subj = 'EM09'
save_data(subj, scores[subj], PERIOD, STAGES, chan_type=chan_type, **options)

INFO:/home/gio/projects/spgr/scripts/spgr/read_data.py:EM09 EM09_eeg_xltek_sessA_d05_11_53_38_scores.xml
INFO:/home/gio/projects/spgr/scripts/spgr/read_data.py:Sampling Frequency 512.0 Hz 
INFO:/home/gio/projects/spgr/scripts/spgr/read_data.py:N Channels in common 77 (only in dataset 53, only in chan pos 0)
DEBUG:/home/gio/projects/spgr/scripts/spgr/read_data.py:Channels: GR3, GR4, GR5, GR6, GR7, GR8, GR11, GR12, GR13, GR14, GR15, GR16, GR19, GR20, GR21, GR22, GR23, GR24, GR27, GR28, GR29, GR30, GR31, GR32, GR34, GR35, GR36, GR37, GR38, GR39, GR40, GR41, GR42, GR43, GR44, GR45, GR46, GR47, GR48, GR49, GR50, GR51, GR52, GR53, GR54, GR55, GR56, GR57, GR58, GR59, GR60, GR61, GR62, GR63, GR64, FGR1, FGR2, FGR3, FGR4, FGR5, FGR6, FGR7, FGR8, FGR10, FGR11, FGR12, FGR13, FGR14, FGR15, FPS1, FPS2, FPS3, FPS4, STS1, STS2, STS3, STS4
INFO:/home/gio/projects/spgr/scripts/spgr/read_data.py:Duration:  67.5 min
/home/gio/tools/venv/lib/python3.4/site-packages/scipy/signal/signaltools.py:1578: Deprec

In [ ]:
for subj in good_subj:
    save_data(subj, scores[subj], PERIOD, STAGES, chan_type=chan_type, **options)

In [ ]:
from phypno.attr import Channels

In [ ]:
from spgr.read_data import _select_channels

In [ ]:
all_chan = Channels('/home/gio/recordings/EM09/doc/elec/EM09_elec_pos-names_sessA.csv')
chan_list = _select_channels('/home/gio/recordings/EM09/doc/elec/EM09_eeg_xltek_sessA_d05_11_53_38_channels.json', 'grid')
chan_in_data = chan_list
chan = all_chan(lambda x: x.label in chan_list)

In [ ]:
from numpy import zeros, where, c_, mean
from numpy.linalg import norm
from scipy.spatial.distance import squareform

max_dist = 15

def _create_bipolar_chan(chan, chan_in_data, max_dist):
    chan = chan(lambda x: x.label in chan_in_data)

    chan_dist = zeros((chan.n_chan, chan.n_chan), dtype='bool')
    for i0, chan0 in enumerate(chan.chan):
        for i1, chan1 in enumerate(chan.chan):
            if i0 < i1 and norm(chan0.xyz - chan1.xyz) < max_dist:
                chan_dist[i0, i1] = True

    x_all, y_all = where(chan_dist)

    bipolar_labels = []
    bipolar_xyz = []
    bipolar_trans = []

    for x0, x1 in zip(x_all, y_all):

        new_label = chan.chan[x0].label + '-' + chan.chan[x1].label
        bipolar_labels.append(new_label)

        xyz = mean(c_[chan.chan[x0].xyz, chan.chan[x1].xyz], axis=1)
        bipolar_xyz.append(xyz)

        trans = zeros(chan.n_chan)
        trans[x0] = 1
        trans[x1] = -1
        bipolar_trans.append(trans)

    bipolar_xyz = c_[bipolar_xyz]
    bipolar_trans = c_[bipolar_trans]

    bipolar = Channels(bipolar_labels, bipolar_xyz)

    return bipolar, bipolar_trans

In [ ]:
v0, v1 = _create_bipolar_chan(chan, chan_in_data, max_dist)

In [ ]:
%gui qt

In [ ]:
from pyqtgraph import ImageView

imv = ImageView()
imv.show()
imv.setImage(v1)

In [ ]:
from phypno.viz import Viz3

In [ ]:
v = Viz3()
v.add_chan(chan)
v.add_chan(v0, color=(1, 0, 0, 1))